##### Resources
- [Qiskit Notebook su HHL](https://qiskit.org/textbook/ch-applications/hhl_tutorial.html)
- ["Quantum algorithm for solving linear systems of equations"](https://arxiv.org/abs/0811.3171)
- ["Step-by-Step HHL Algorithm Walkthrough to Enhance the Understanding of Critical Quantum Computing Concepts"](https://arxiv.org/abs/2108.09004)
- [Notes on Quantum Computing](https://homepages.cwi.nl/~rdewolf/qcnotes.pdf)
- [Quantum Algorithm Zoo](https://quantumalgorithmzoo.org/)

#### HHL Algorithm (Qiskit Notebook)

Data una matrice $A\in \mathbb{C}^{N\times N}$ e un vettore $\overrightarrow{b}\in\mathbb{C}^{N}$ trovare $\overrightarrow{x}\in\mathbb{C}^{N}$ che soddisfa $A\overrightarrow{x} = \overrightarrow{b}$

Un sistema di equazioni lineari è detto $s$-sparso se $A$ ha al più $s$ elementi $\neq 0$ per colonna o riga.

Dato $\mathbb{k}$ numero di condizionamento del sistema e $\epsilon$ accuratezza, risolvere un sistema $s$-sparso di dimensione $N$ su un computer classico richiede $O(Ns\mathbb{k}\log\left(\frac{1}{\epsilon}\right))$

L'algoritmo HHL richiede tempo $O\left(\frac{\displaystyle \log(N)s^2\mathbb{k}^2}{\displaystyle \epsilon}\right)$ per **stimare una funzione della soluzione** quando $A$ è una matrice Hermitiana (quindi quando $A=\overline{A^T}=A^*$, cioè $A$ uguale alla sua coniugata trasposta), sotto l'assunzione di avere oracoli efficienti per caricare i dati, simulazione Hamiltoniana. Lo speedup è esponenziale nella dimensione del sistema, con una differenza importante: l'algoritmo classico ritorna la soluzione completa, mentre HHL può solo approssimare funzioni del vettore soluzione.

Simulazione Hamiltoniana riguarda la ricerca della complessità computazione. Data $H$ Hamiltoniana (cioè rappresentante l'energia totale del sistema quantico), un tempo $t$ e un errore di simulazione massimo $\epsilon$, l'obiettivo di una simulazione Hamiltoniana è trovare un algoritmo che approssimi $U$ tale che $\|U-e^{\displaystyle -iHt}\|\leq\epsilon$ dove $e^{-iHt}$ è l'evoluzione ideale e $\|A\|$ è la norma spettrale (la radice dell'autovalore di modulo massimo di $A^*A$)

##### Codificare il problema nel mondo quantico

Riscalando il sistema, possiamo assumere $\overrightarrow{b}$ e $\overrightarrow{x}$ normalizzati e mapparli negli stati quantici rispettivi $\ket{b}$ e $\ket{x}$. Solitamente il mapping è tale che l'$i$-esimo componente di $\overrightarrow{b}$ corrisponde all'ampiezza dell'$i$-esimo basis state dello stato quantico $\ket{b}$, idem per $\overrightarrow{x}$ e $\ket{x}$

Da qui in poi consideriamo il problema riscalato $$A\ket{x} = \ket{b}$$
Dato che $A$ è Hermitiana, che significa che è una matrice quadrata complessa uguale alla sua coniugata trasposta (cioè $a_{ij} = \overline{a_ij} \Leftrightarrow A = \overline{A^T} \Leftrightarrow A = A^H = A^*$), allora ha una decomposizione spettrale $$A=\sum_{j=0}^{N-1} \lambda_j\ket{u_j}\bra{u_j}$$ con $\lambda_j\in\mathbb{R}$ e $\ket{u_j}$ il $j$-esimo autovettore di $A$ rispetto all'autovalore $\lambda_j$. Questo porta a $$A^{-1} = \sum_{j=0}^{N-1}\lambda_j^{-1}\ket{u_j}\bra{u_j}$$ Quindi la parte destra del sistema può essere riscritta in termini della base di autovettori di $A$ come $$\ket{b} = \sum_{j=1}^{N-1}b_j\ket{u_j}$$ con $b_j\in\mathbb{C}$

Ricordare che l'obiettivo di HHL è terminare l'algoritmo con il registro di readout nello stato $$\ket{x} = A^{-1}\ket{b} = \sum_{j=0}^{N-1}\lambda_j^{-1}b_j\ket{u_j}$$

Le costanti di normalizzazione sono implicite, dato che parliamo di stati quantici.

##### Descrizione dell'Algoritmo HHL

L'algoritmo usa 3 registri quantici, tutti settati a $\ket{0}$ all'inizio.
- $n_l$ è usato per memorizzare la rappresentazione binaria degli autovalori di $A$
- $n_b$ contiene il vettore soluzione ($N = 2^{n_b}$)
- Un registro extra per i qubit ausiliari, usati come passaggi intermedi nelle computazioni singole, ignorati perché settati a $\ket{0}$ all'inizio e risettati a $\ket{0}$ alla fine della singola operazione

![circuit](https://qiskit.org/textbook/ch-applications/images/hhlcircuit.png)

1. Carica $\ket{b}\in\mathbb{C}^N$, cioè esegue la trasformazione $\ket{0}_{n_b}\mapsto\ket{b}_{n_b}$
2. Applica QPE (Quantum Phase Estimation) con $$U = e^{iAt} = \sum_{j=0}^{N-1}e^{i\lambda_j t}\ket{u_j}\bra{u_j}$$
Lo stato quantico del registro espresso nella base di autovalori di $A$ è ora $$\sum_{j=0}^{N-1}b_j\ket{\lambda_j}_{n_l}\ket{u_j}_{n_b}$$ con $\ket{\lambda_j}_{n_l}$ la rappresentazione binaria di $\lambda_j$ su $n_l$ bit
3. Aggiungi un qubit ausiliario e applica la rotazione condizionata su $\ket{\lambda_j}$ $$\sum_{j=0}^{N-1}b_j\ket{\lambda_j}_{n_l}\ket{u_j}_{n_b}\left(\sqrt{1-\frac{C^2}{\lambda_j^2}}\ket{0}+\frac{C}{\lambda_j}\ket{1}\right)$$ dove $C$ è una costante di normalizzazione e, espressa come sopra, dovrebbe essere $|C| < \lambda_{\min}$
4. Applica QPE $^T$, e ignorando i possibili errori risulterà  in $$\sum_{j=0}^{N-1}b_j\ket{0}_{n_l}\ket{u_j}_{n_b}\left(\sqrt{1-\frac{C^2}{\lambda_j^2}}\ket{0}+\frac{C}{\lambda_j}\ket{1}\right)$$ 
5. Misura il bit ausiliario nella base computazionale. Se è 1, allora il registro è nello stato di post-misurazione $$\left(\sqrt{\frac{1}{\sum_{j=0}^{N-1}\frac{|b_j|^2}{|\lambda_j|^2}}}\right)\sum_{j=0}^{N-1}\frac{b_j}{\lambda_j}\ket{0}_{n_1}\ket{u_j}_{n_b}$$ corrispondente a soluzione a meno di un fattore di normalizzazione.
6. Applica un osservabile $M$ per calcolare $F(x) = \langle x\:|\:M\:|\:x\rangle$

##### QPE in HHL

QPE è un algoritmo quantico che, data una matrice unitaria $U$ con autovettore $\ket{\psi}$ e autovalore $e^{2\pi i\theta}$, trova $\theta$. La definizione formale è quella che segue:

Data $U\in\mathbb{C}^{2^m\times 2^m}$ unitaria e $\ket{\psi}_m\in \mathbb{C}^{2^m}$ uno dei suoi autovettori di autovalore $e^{2\pi i\theta}$, QPE prende in input $U$ e lo stato $\ket{0}_n\ket{\psi}_m$ e ritorna lo stato $\ket{\overline{\theta}}_n\ket{\psi}_m$ con $\overline{\theta}$ approssimazione binaria di $2^n\theta$ e $_n$ a indicare il troncamento a $n$ cifre.
$$QPE(U,\ket{0}_n\ket{\psi}_m)=\ket{\overline{\theta}}_n\ket{\psi}_m$$

Per HHL useremo QPE con $U = e^{iAt}$, dove $A$ è la matrice associata al sistema che vogliamo risolvere. In questo caso:
$$e^{iAt}=\sum_{j=0}^{N-1}e^{i\lambda_j t}\ket{u_j}\bra{u_j}$$

Per l'autovettore $\ket{u_j}_{n_b}$, che ha autovalore $e^{i\lambda_j t}$, QPE emetterà in output $\ket{\overline{\lambda}_j}_{n_l}\ket{u_j}_{n_b}$. $\overline{\lambda}_j$ rappresenta una rappresentazione binaria su $n_l$ bit di $2^{n_l}\frac{\lambda_jt}{2\pi}$. Quindi, se ogni $\lambda_j$ può essere rappresentato esattamente su $n_l$ bits, abbiamo
$$QPE\left(e^{iAt},\sum_{j=0}^{N-1}b_j\ket{0}_{n_l}\ket{u_j}_{n_b}\right)=\sum_{j=0}^{N-1}b_j\ket{\lambda_j}_{n_l}\ket{u_j}_{n_b}$$

In realtà, lo stato quantico nel registro dopo aver applicato QPE allo stato iniziale è $$\sum_{j=0}^{N-1}b_j\left(\sum_{l=0}^{2^{n_l}-1}\alpha_{l|j}\ket{l}_{n_l}\right)\ket{u_j}_{n_b}$$
con $$\alpha_{l|j} = \frac{1}{2^{n_l}}\sum_{k=0}^{2^{n_l}-1}\left(e^{2\pi i\left(\frac{\lambda_j t}{2\pi}-\frac{l}{2^{n_l}}\right)}\right)^k$$
Denotiamo con $\overline{\lambda_j}$ la migliore approssimazione su $n_l$ bit di $\lambda_j$ con $1\leq j\leq N$, possiamo ridefinire il registro $n_l$ così che $\alpha_{l|j}$ denota l'ampiezza di $\ket{l+\overline{\lambda}_j}_{n_l}$
$$\alpha_{l|j} = \frac{1}{2^{n_l}} \sum_{k=0}^{2^{n_l}-1} \left(e^{2\pi i\left(\frac{\lambda_j t}{2\pi}-\frac{l+\overline{\lambda}_j}{2^{n_l}}\right)}\right)^k$$

Se ogni $\frac{\lambda_jt}{2\pi}$ può essere rappresentato esattamente su $n_l$ bits, allora $\frac{\lambda_jt}{2\pi} = \frac{\overline{\lambda}_j}{2^{n_l}}$ per ogni $j$. Quindi in questo caso $\forall\:j\:\:1\leq j\leq N$ abbiamo che $\alpha_{0|j}=1$ e $\alpha_{l|j} = 0\:\:\forall\:l\neq 0$, solo in questo caso possiamo scrivere che lo stato del registro dopo QPE è $$\sum_{j=0}^{N-1}b_j\ket{\lambda_j}_{n_l}\ket{u_j}_{n_b}$$
Altrimenti $|\alpha_{l|j}|$ è grande $\Leftrightarrow \frac{\lambda_jt}{2\pi} \simeq \frac{l+\overline{\lambda}_j}{2^{n_l}}$ e lo stato del registro è
$$\sum_{j=0}^{N-1}\sum_{l=0}^{2^{n_l}-1}\alpha_{l|j}b_j\ket{l}_{n_l}\ket{u_j}_{n_b}$$

##### Esempio con 4 qubits

$$A=\left(\begin{array}{cc}1&-\frac{1}{3}\\-\frac{1}{3}&1\end{array}\right)\:\:\ket{b}=\left(\begin{array}{c}1\\0\end{array}\right)$$

Useremo $n_b=1$ per rappresentare $\ket{b}$ e la soluzione $\ket{x}$, $n_l=2$ qubits per memorizzare la rappresentazione binaria degli autovalori a $1$ qubit ausiliario per memorizzare se la rotazione condizionata, e quindi l'algoritmo, ha avuto successo o no.

Per poter mostrare meglio l'algoritmo calcoliamo gli autovalori di $A$ in modo da scegliere $t$ così da ottenere una rappresentazione binaria esatta degli autovalori riscalati nel registro $n_l$. Notare che nell'implementazione dell'algoritmo HHL non si ha bisogno di conoscenza pregressa sugli autovalori.
$$\lambda_1=\frac{2}{3}\:\:\lambda_2=\frac{4}{3}$$
QPE emetterà un'approssimazione binaria su $n_l$ bit ($2$ in questo esempio) di $\frac{\lambda_jt}{2\pi}$, quindi impostando $t=2\pi\cdot\frac{3}{8}$ QPE darà un'approssimazione binaria su $2$ bit di $$\frac{\lambda_1t}{2\pi}=\frac{1}{4}\:\:\frac{\lambda_2t}{2\pi}=\frac{1}{2}$$
che sono rispettivamente $$\ket{01}_{n_1}\:\:\ket{10}_{n_l}$$
dai rispettivi autovettori $$\ket{u_1}\frac{1}{\sqrt{2}}\left(\begin{array}{c}1\\-1\end{array}\right)\:\:\ket{u_2}\frac{1}{\sqrt{2}}\left(\begin{array}{c}1\\1\end{array}\right)$$
Una generica matrice Hermitiana $A$ di dimensione $N$ può avere fino a $N$ autovalori distinti, richiedendo così $O(N)$ per il calcolo e il vantaggio quantistico verrebbe perso.

Possiamo riscrivere $\ket{b}$ in termini della base di autovalori di $A$ come $$\ket{b}_{n_b} = \sum_{j=1}^2\frac{1}{\sqrt{2}}\ket{u_j}_{n_b}$$

Ora vediamo i diversi passaggi dell'algoritmo:

1. La preparazione dello stato è triviale, dato che $\ket{b} = \ket{0}$
2. Applicare QPE darà $$\frac{1}{\sqrt{2}}\ket{01}\ket{u_1} + \frac{1}{\sqrt{2}}\ket{10}\ket{u_2}$$
3. Rotazione condizionata con $C=\frac{1}{8}$ che è minore dell'autovalore minimo (riscalato) pari a $\frac{1}{4}$. Notare che la costante $C$ qui deve essere scelta in modo che sia minore dell'autovalore minimo (riscalato) ma il più grande possibile così che quando il qubit ausiliario è misurato la probabilità di essere nello stato $\ket{1}$ è grande.
$$\frac{1}{\sqrt{2}}\ket{01}\ket{u_1}\left(\sqrt{1-\frac{\left(\frac{1}{8}\right)^2}{\left(\frac{1}{4}\right)^2}}\ket{0} + \frac{\frac{1}{8}}{\frac{1}{4}}\ket{1}\right) + \frac{1}{\sqrt{2}}\ket{10}\ket{u_2}\left(\sqrt{1-\frac{\left(\frac{1}{8}\right)^2}{\left(\frac{1}{2}\right)^2}}\ket{0} + \frac{\frac{1}{8}}{\frac{1}{2}}\ket{1}\right) =$$ 
$$= \frac{1}{\sqrt{2}}\ket{01}\ket{u_1}\left(\sqrt{1-\frac{1}{4}}\ket{0} + \frac{1}{2}\ket{1}\right) + \frac{1}{\sqrt{2}}\ket{10}\ket{u_2}\left(\sqrt{1-\frac{1}{16}}\ket{0} + \frac{1}{4}\ket{1}\right)$$
4. Dopo aver applicato QPE $^T$ il quantum computer è nello stato
$$\frac{1}{\sqrt{2}}\ket{00}\ket{u_1}\left(\sqrt{1-\frac{1}{4}}\ket{0} + \frac{1}{2}\ket{1}\right) + \frac{1}{\sqrt{2}}\ket{00}\ket{u_2}\left(\sqrt{1-\frac{1}{16}}\ket{0} + \frac{1}{4}\ket{1}\right)$$
5. Sul risultato $1$, quando misuriamo in qubit ausiliario, lo stato è
$$\frac{\frac{1}{\sqrt{2}}\ket{00}\ket{u_1}\frac{1}{2}\ket{1} + \frac{1}{\sqrt{2}}\ket{00}\ket{u_2}\frac{1}{4}\ket{1}}{\sqrt{\frac{5}{32}}}$$
E possiamo notare che $$\frac{\frac{1}{2\sqrt{2}}\ket{u_1} + \frac{1}{4\sqrt{2}}\ket{u_2}}{\sqrt{\frac{5}{32}}} = \frac{\ket{x}}{\|x\|}$$
6. Senza usare altri gate, possiamo calcolare la norma di $\ket{x}$: è la probabilità di misurare $1$ nel bit ausiliario dal precedente step
$$P(\ket{1}) = \left(\frac{1}{2\sqrt{2}}\right)^2 + \left(\frac{2}{4\sqrt{2}}\right)^2 = \frac{5}{32}

##### Implementazione in Qiskit

Qiskit fornisce già un'implementazione dell'algoritmo HHL che richiede solo la matrice $A$ e $\ket{b}$ come input. Possiamo dare all'algoritmo una matrice Hermitiana generica e uno stato iniziale arbitrario come array NumPy, ma non otterrà uno speedup esponenziale perché l'implementazione di default è esatta e quindi esponenziale nel numero di qubit (non c'è nessuno algoritmo in grado di preparare esattamente un stato quantico arbitrario usando risorse polinomiali nel numero di qubit, o che può eseguire esattamente l'operazione $e^{iAt}$ per qualche matrice Hermitiana generica $A$ usando risorse polinomiali nel numero di qubit). Se si conoscono implementazioni efficienti per un particolare problema, la matrice e/o il vettore possono essere forniti come oggetti `QuantumCircuit`.

L'interfaccia per gli algoritmi che risolvono sistemi lineari è `LinearSolver`, ed il problema viene specificato quando si chiama il metodo `solve()`.

L'implementazione più semplice prende la matrice e il vettore come array NumPy, e di seguito creiamo anche un `NumPyLinearSolver` (l'algoritmo classico) per validare le nostre soluzioni.

In [3]:
import numpy as np
from qiskit.algorithms.linear_solvers.numpy_linear_solver import NumPyLinearSolver
from qiskit.algorithms.linear_solvers.hhl import HHL


A = np.array([[1,-1/3], [-1/3, 1]])
b = np.array([1, 0])
naive_hhl_sol = HHL().solve(A, b)

Per il risolutore classico dobbiamo riscalare il vettore (cioè `b / np.linal.norm(vector)`) per considerare la rinormalizzazione che avviene quando `b` è codificato in uno stato quantico in HHL

In [4]:
classical_sol = NumPyLinearSolver().solve(A, b/np.linalg.norm(b))

Il package `linear_solvers` contiene una cartella chiamata `matrices` che funge da placeholder per implementazioni efficienti di tipologie particolari di matrici. Allo stato attuale contiene solo l'implementazione efficiente (cioè polinomiale nel numero di qubit) di `TridiagonalToeplitz`: matrici reali simmetriche nella forma
$$A=\left(\begin{array}{c c c c}a&b&0&0\\ b&a&b&0\\ 0&b&a&b\\ 0&0&b&a \end{array}\right)$$
$$a,b \in \mathbb{R}$$

Non si considerano matrici non simmetriche poiché HHL assume matrici Hermitiane, cioè $A = \overline{A^T}$

Dato che la matrice dell'esempio è in questa forma possiamo usare un'istanza di `TridiagonalToeplitz(n_qubits, a, b)` e comparare i risultati rispetto al risolvere il sistema con un array come input.

In [5]:
from qiskit.algorithms.linear_solvers.matrices.tridiagonal_toeplitz import TridiagonalToeplitz


tridi_A = TridiagonalToeplitz(1, 1, -1/3)
tridi_sol = HHL().solve(tridi_A, b)

Ricordiamo che HHL trova una soluzione esponenzialmente più veloce nella dimensione del sistema rispetto alla controparte classica, ma il prezzo da pagare è che non si ottiene una soluzione esatta ma uno stato quantico che rappresenta il vettore $x$, e apprendere tutte le componenti di questo vettore richiederebbe un tempo lineare nella dimensione, diminuendo lo speedup guadagnato.

Quindi possiamo solo calcolare funzioni di $x$ (i così chiamati **observables**) per apprendere informazioni sulla soluzione. Questo è riflesso in `LinearSolverResult` ritornato da `solve()`, che contiene le seguenti proprietà:
- `state`: il circuito che prepara la soluzione o la soluzione come vettore
- `euclidean_norm`: la norma Euclidea se l'algoritmo sa come calcolarla
- `observable`: la lista degli observable calcolati
- `circuit_results`: i risultati osservabili delle lista dei circuiti

Vedimo le soluzioni ottenute. `classical_solution` è il risultato dell'algoritmo classico, quindi chiamando `state` otteniamo un array:

In [6]:
print('Classical state:', classical_sol.state)

Classical state: [1.125 0.375]


Gli altri due esempi erano algoritmi quantici, quindi possiamo solo vedere lo stato quantico attraverso il circuito quantico che prepara lo stato soluzione:

In [7]:
print("Naive state")
print(naive_hhl_sol.state)

Naive state
      ┌────────────┐┌──────┐        ┌─────────┐
  q4: ┤ circuit-85 ├┤3     ├────────┤3        ├
      └────────────┘│      │┌──────┐│         │
q5_0: ──────────────┤0     ├┤2     ├┤0        ├
                    │  QPE ││      ││  QPE_dg │
q5_1: ──────────────┤1     ├┤1     ├┤1        ├
                    │      ││  1/x ││         │
q5_2: ──────────────┤2     ├┤0     ├┤2        ├
                    └──────┘│      │└─────────┘
  q6: ──────────────────────┤3     ├───────────
                            └──────┘           


In [8]:
print("Tridiagonal state")
print(tridi_sol.state)

Tridiagonal state
       ┌─────────────┐┌──────┐        ┌─────────┐
  q26: ┤ circuit-298 ├┤3     ├────────┤3        ├
       └─────────────┘│      │┌──────┐│         │
q27_0: ───────────────┤0     ├┤2     ├┤0        ├
                      │  QPE ││      ││  QPE_dg │
q27_1: ───────────────┤1     ├┤1     ├┤1        ├
                      │      ││  1/x ││         │
q27_2: ───────────────┤2     ├┤0     ├┤2        ├
                      └──────┘│      │└─────────┘
  q28: ───────────────────────┤3     ├───────────
                              └──────┘           


Ricordando che la norma Euclidea di un vettore $x=(x_1,\ldots,x_N)$ è definita come $\|x\| = \sqrt{\sum_{i=1}^N x_i^2}$, la probabilità di misurare $1$ nel qubit ausiliario nello step 5. è la norma al quadrato di $x$. Questo significa che l'algoritmo HHL può sempre calcolare la norma euclidea della soluzione e possiamo comprare l'accuratezza dei risultati:

In [9]:
print("Classical Euclidean norm:", classical_sol.euclidean_norm)
print("Naive Euclidean norm:", naive_hhl_sol.euclidean_norm)
print("Tridiagonal Euclidean norm:", tridi_sol.euclidean_norm)

Classical Euclidean norm: 1.1858541225631423
Naive Euclidean norm: 1.1858541225631407
Tridiagonal Euclidean norm: 1.185854122563139


Comparare i vettori soluzioni componente per componente è più complesso, di nuovo riflettendo l'idea che non possiamo ottenere il vettore soluzione completo dall'algoritmo quantico. Per didattica, possiamo verificare che effettivamente i diversi vettori soluzione ottenuti sono buone approssimazioni anche a livello di componente.

Per fare ciò, prima dobbiamo usare `StateVector` dal package `quantum_info` e estrarre i componenti del vettore di destra, cioè quelli corrispondenti al qubit ancillare (in basso nei circuiti) con valore $1$ e ai qubit di lavoro (i due nel mezzo nei circuiti) con valore $0$. Quindi siamo interessati agli stati $1000$ e $1001$, corrispondneti rispettivamente al primo e secondo componente del vettore soluzione.

In [10]:
from qiskit.quantum_info import Statevector


naive_sv = Statevector(naive_hhl_sol.state).data
tridi_sv = Statevector(tridi_sol.state).data

# Extract the right component: 1000 -> index 8, 1001 -> index 9
naive_fullvec = np.array([naive_sv[8], naive_sv[9]])
tridi_fullvec = np.array([tridi_sv[8], tridi_sv[9]])

print("Naive raw solution vector:", naive_fullvec)
print("Tridi raw solution vector:", tridi_fullvec)

Naive raw solution vector: [ 5.74337504e-17+8.93013426e-18j -1.12425450e-17+5.23599292e-16j]
Tridi raw solution vector: [-1.92535646e-16+9.81307787e-17j -2.24504108e-17+6.80168960e-16j]


I componenti immaginari potrebbero far pensare che i risultati siano sbagliati ma essendo molto molto piccoli sono probabilmente dovuti alla precisione del computer, possono essere ignorati:

In [11]:
naive_fullvec = np.real(naive_fullvec)
tridi_fullvec = np.real(tridi_fullvec)

Successivamente si dividono i vettore per le rispettive norme, così da eliminare le costanti provenienti dalle differenti parti del circuito. I vettori soluzione completi possono essere recuperati moltiplicando questi vettori normalizzati per le rispettive norme euclidee calcolate prima.

In [12]:
print('Full naive solution vector:', naive_hhl_sol.euclidean_norm*naive_fullvec/np.linalg.norm(naive_fullvec))
print('Full tridi solution vector:', tridi_sol.euclidean_norm*tridi_fullvec/np.linalg.norm(tridi_fullvec))
print('Classical state:', classical_sol.state)

Full naive solution vector: [ 1.16376749 -0.22780523]
Full tridi solution vector: [-1.17787369 -0.13734469]
Classical state: [1.125 0.375]


`naive_hhl_sol` *dovrebbe* essere esatta perché tutti i metodi utilizzati di default sono esatti.

`tridi_sol` è esatta solo nel caso di sistemi $2\times 2$, e per matrici più grandi sarà un'approssimazione come mostrato nell'esempio seguente:

In [13]:
from scipy.sparse import diags


n_qubits = 2
matrix_size = 2 ** n_qubits
# valori della matrice simmetrica tridiagonale Toeplitz
elem_a = 1
elem_b = -1/3

A = diags([elem_b, elem_a, elem_b], [-1, 0, 1], shape=(matrix_size, matrix_size)).toarray()
b = np.array([1] + [0]*(matrix_size - 1))

# esecuzione degli algoritmi
classical_sol = NumPyLinearSolver().solve(A, b/np.linalg.norm(b))
naive_hhl_sol = HHL().solve(A, b)
A_tridi = TridiagonalToeplitz(n_qubits, elem_a, elem_b)
tridi_sol = HHL().solve(A_tridi, b)

print("Classical Euclidean norm:", classical_sol.euclidean_norm)
print("Naive Euclidean norm:", naive_hhl_sol.euclidean_norm)
print("Tridiagonal Euclidean norm:", tridi_sol.euclidean_norm)

Classical Euclidean norm: 1.237833351044751
Naive Euclidean norm: 1.2099806231119126
Tridiagonal Euclidean norm: 1.209457721870593


Possiamo anche comparare la differenza in termini di risorse, fra il metodo esatto e l'implementazione efficiente. La dimensione di $2\times 2$ del sistema è ancora speciale poiché richiede all'algoritmo esatto meno risorse, ma aumentando la dimensione possiamo vedere che il metodo esatto scala le risorse esponenzialmente nel numero di qubit mentre `TridiagonalToeplitz` è polinomiale.

In [14]:
from qiskit import transpile


n_qubits = list(range(1, 5))
elem_a = 1
elem_b = -1/3

i = 1
# calcola la profondità dei circuiti per diversi n_qubit per comparare l'uso di risorse
naive_depths = []
tridi_depths = []

for nq in n_qubits:
    A = diags([elem_b, elem_a, elem_b], [-1, 0, 1], shape = (2**nq, 2**nq)).toarray()
    b = np.array([1] + [0]*(2**nq - 1))

    naive_hhl_sol = HHL().solve(A, b)
    A_tridi = TridiagonalToeplitz(nq, elem_a, elem_b)
    tridi_sol = HHL().solve(A_tridi, b)

    naive_circuit = transpile(naive_hhl_sol.state, basis_gates = ['id', 'rz', 'sx', 'x', 'cx'])
    tridi_circuit = transpile(tridi_sol.state, basis_gates = ['id', 'rz', 'sx', 'x', 'cx'])

    naive_depths.append(naive_circuit.depth())
    tridi_depths.append(tridi_circuit.depth())
    i += 1

sizes = [str(2**nq) + "x" + str(2**nq) for nq in n_qubits]
columns = ["Dim. del sistema", "Profondità naive_sol", "Profondità tridi_sol"]
data = np.array([sizes, naive_depths, tridi_depths])
row_format = "{:>23}" * (len(columns) + 2)

for column, row in zip(columns, data):
    print(row_format.format(column, *row))

       Dim. del sistema                    2x2                    4x4                    8x8                  16x16
   Profondità naive_sol                    334                   2578                  33141                 404340
   Profondità tridi_sol                    565                   5107                  14756                  46552


Il motivo per cui l'implementazione sembra aver bisogno comunque di risorse esponenziali è perché l'attuale implementazione della rotazione condizionata è esatta (passaggio 3.) per cui richiede risorse esponenziali in $n_l$. Possiamo calcolare quante risorse in più richiede l'implementazione di default comparata al Tridiagonal, dato che si differenziano solo in come implementano $e^{iAt}$

In [15]:
print("Eccesso:", [naive_depths[i] - tridi_depths[i] for i in range(len(naive_depths))])

Eccesso: [-231, -2529, 18385, 357788]


Ora vediamo cosa contengono `observables` e `circuit_results`. Per calcolare funzioni del vettore soluzione $x$ possiamo fornire al metodo `solve()` un `LinearSystemObservable` come input. Ce ne sono due tipi disponibili:

In [16]:
from qiskit.algorithms.linear_solvers.observables import AbsoluteAverage, MatrixFunctional

Per un vettore $x = (x_1,\ldots,x_N)$ l'observable `AbsoluteAverage` calcola $|\frac{1}{N}\sum_{i=1}^N x_i|$

In [17]:
n_qubits = 1
matrix_size = 2 ** n_qubits
elem_a = 1
elem_b = -1/3

A = diags([elem_b, elem_a, elem_b], [-1, 0, 1], shape=(matrix_size, matrix_size)).toarray()
b = np.array([1] + [0]*(matrix_size - 1))
A_tridi = TridiagonalToeplitz(1, elem_a, elem_b)

average_sol = HHL().solve(A_tridi, b, AbsoluteAverage())
classical_average = NumPyLinearSolver().solve(A, b/np.linalg.norm(b), AbsoluteAverage())

print("Quantum average:", average_sol.observable)
print("Classical average:", classical_average.observable)
print("Quantum circuit results", average_sol.circuit_results)

Quantum average: 0.7499999999999976
Classical average: 0.75
Quantum circuit results (0.49999999999999706+0j)


L'observable `MatrixFunctional` calcola $x^TBx$ per un vettore $x$ a una matrice tridiagonale simmetrica Toeplitz $B$. La classe prende i valori della diagonale principale e secondaria nel suo costruttore.

In [18]:
observable = MatrixFunctional(1, 1/2)

functional_sol = HHL().solve(A_tridi, b, observable)
classical_functional = NumPyLinearSolver().solve(A, b/np.linalg.norm(b), observable)

print('Quantum functional:', functional_sol.observable)
print('Classical functional:', classical_functional.observable)
print('Quantum circuit results:', functional_sol.circuit_results)

Quantum functional: 1.8281249999999882
Classical functional: 1.828125
Quantum circuit results: [(0.6249999999999963+0j), (0.49999999999999706+0j), (0.12499999999999926+0j)]


Quindi `observable` conteien il valore finale della funzione applicata a $x$, mentre `circuit_result` contiene i valori crudi ottenuti dal circuito e usati per processare il risultato dell'observable.

Il "come processare il risultato" è meglio spiegato guardando gli argomenti che `solve()` richiede: ne accetta fino a 5

- `matrix: Union[np.ndarray, QuantumCircuit]`, la matriche che definisce il sistema lineare
- `vector: Union[np.ndarray, QuantumCircuit]`, il vettore alla destra nell'equazione
- `observable: Optional[Union[LinearSystemObservable, BaseOperator, List[BaseOperator]]] = None`, l'observable o l'eventuale lista di observable da calcolare sul vettore soluzione $x$, specificabile in due diversi modi: un'opzione è di fornire una lista di `LinearSystemObservable` come terzo e ultimo parametro, alternativamente possiamo fornire la nostra implementazione di `observable`, `post_rotation` e `post_processing` (gli ultimi due parametri) dove:
    - `observable` è l'operatore che deve calcolare il valore atteso dell'observable, ad esempio un `PauliSumOp`
    - `post_rotation: Optional[Union[QuantumCircuit, List[QuantumCircuit]]] = None` è il circuito da applicare alla soluzione per estrarne informazioni se sono necessari ulteriori gate
    - `post_processing: Optional[Callable[[Union[float, List[float]]], Union[float, List[float]]]] = None` è la funzione per calcolare il valore degli observable dalle probabilità calcolate

In altri termini ci saranno tanti circuiti `circuit_results` quanti circuiti `post_rotation`, e `post_processing` dice all'algoritmo come usare i valori che vediamo quando stampiamo `circuit_results` per ottenere i valure che vediamo quando stampiamo `observables`.

Infine, la classe `HHL()` accetta i seguenti parametri nel suo costruttore:
- Tolleranza dell'errore, accuratezza dell'approssimazione di default a $1e-2$
- Expectation, come vengono calcolati i valori attesi (default: `PauliExpectation`)
- Quantum instance: il `QuantumInstance` o backend, di default è una simulazione `Statevector`

In [19]:
from qiskit import Aer

backend = Aer.get_backend('aer_simulator')
hhl = HHL(1e-3, quantum_instance=backend)

accurate_solution = hhl.solve(A, b)
classical_solution = NumPyLinearSolver().solve(A, b / np.linalg.norm(b))

print(accurate_solution.euclidean_norm)
print(classical_solution.euclidean_norm)

1.1858541225631407
1.1858541225631423


#### *Quantum algorithm for linear systems of equations*, Harrow, Hassidim, Lloyd

This algorithm yields an approximation of $x$ for $Ax=b$, useful to compute functions on $x$ like $x^TMx$, in time polynomial in $\log N, k$ with $A$ of $N\times N$ with conditioning number $\mathbb{k}$ (ratio between the largest and smallest eigenvalues)

Data una matrice Hermitiana $A$ $N\times N$ e un vettore unitario $b$, supponiamo di voler trovare $x$ tale che $Ax = b$.
- L'algoritmo rappresenta $b$ come uno stato quantico $\ket{b} = \sum_{i=1}^N b_i\ket{i}$
- Usando tecniche di simulazione Hamiltoniana si applica $e^{iAt}$ a $\ket{b}$ per una superposizione di tempi $t$ diversi. Quest'abilità di esponenziare $A$ si traduce, con la ben conosciuta QPE, nell'abilità di decomporre $\ket{b}$ in una base di autovalori di $A$ e di trovare i corrispondenti autovalori $\lambda_j$
- Informalmente, lo stato attuale del sistema sarà vicino a $\sum_{j=1}^N\beta_j\ket{u_j}\ket{\lambda_j}$, con $u_j$ autovalore della base di $A$ e $\ket{b} = \sum_{i=1}^N \beta_i\ket{u_i}$
- Quindi eseguiamo una mappatura lineare portando $\ket{\lambda_j}$ in $C\lambda_j^{-1}\ket{\lambda_j}$, con $C$ costante di normalizzazione. Quest'operazione non è unitaria, ha una probabilità di fallire che verrà trattata in seguito.
- Quando ha successo, calcoliamo il registro $\ket{\lambda_j}$ e rimaniamo con uno stato proporzionale a $\sum_{j=1}^N\beta_j\lambda_j^{-1}\ket{u_j} = A^{-1}\ket{b} = \ket{x}$

Con il crescere di $\mathbb{k} = \frac{\lambda_{\max}}{\lambda_{\min}}$, il numero di condizionamento, $A$ diventa più simile ad una matrice non invertibile e le soluzioni diventano meno stabili. Una tale matrice è definita "mal-condizionata".

L'algoritmo assume che i valori singolari di $A$ siano fra $\frac{1}{\mathbb{k}}$ e $1$, equivalentamente $\mathbb{k}^{-2}I\leq A^TA\leq I$. In questo caso il tempo d'esecuzione scala con $\frac{\mathbb{k}^2\log(N)}{\epsilon}$ con $\epsilon$ errore additivo ottenuto nello stato di output $\ket{x}$. Quindi con $k$ e $\frac{1}{\epsilon}$ entrambi polinomiali in $\log(N)$, l'algoritmo ottiene uno speedup esponenziale.

##### L'algoritmo nel Dettaglio

Si vuole trasformare una matrice Hermitiana $A$ in un operatore unitario $e^{iAt}$ da applicare a volontà. Questo è possibile, tra gli altri casi, se $A$ è $s$-sparsa a livello di riga, cioè ha al più $s$ elementi $\neq 0$ per riga, e efficientemente computabile per riga, cioè che dato un indice di riga questi valori non-0 possono essere calcolati in $O(s)$. Sotto queste assunzioni, si può simulare $e^{iAt}$ in $\overline{O}(\log(N)s^2t)$ con $\overline{O}$ che esclude i termini che crescono più lentamente.

Se $A$ non è Hermitiana, si definisce $$C=\left(\begin{array}{cc}0&A\\A^T&0\end{array}\right)$$ Dato che $C$ è Hermitiana, possiamo risolvere l'equazione $Cy = \left(\begin{array}{c}b\\0\end{array}\right)$ ottenendo $y=\left(\begin{array}{c}0\\x\end{array}\right)$. Quindi possiamo applicare questa riduzione se necessario, nel caso di $A$ non Hermitiana. Da qui in poi assumiamo $A$ come Hermitiana.

Si necessita anche di una procedura efficiente per preparare $\ket{b}$.

Il prossimo passo è decomporre $\ket{b}$ nella base di autovettori, usando phase estimation. Indichiamo con $\ket{u_j}$ gli autovettori di $A$ (o equivalentemente di $e^{iAt}$) e con $\lambda_j$ i rispettivi autovettori. Sia $$\ket{\Psi_0}=\sqrt{\frac{2}{T}}\sum_{\tau=0}^{T-1}\frac{\pi(\tau+\frac{1}{2})}{T}\ket{\tau}$$ per qualche $T$ grande. I coefficienti di $\ket{\Psi_0}$ sono scelti in modo da minimizzare una certa funzione di loss quadratica che appare nell'analisi dell'errore.

Successivamente si applica l'evoluzione Hermitiana condizionale $\sum_{\tau=0}^{T-1}\ket{\tau}\bra{\tau}^C\otimes e^{\frac{iA\tau t_0}{T}}$ a $\ket{\Psi_0}^C\otimes\ket{b}$, dove $t_0=O(\frac{\mathbb{k}}{\epsilon})$

La trasformata di Fourier del primo registro fornisce lo stato
$$\sum_{j=1}^N\sum_{k=0}^{T-1} \alpha_{k|j}\beta_j\ket{k}\ket{u_j}$$
dove $\ket{k}$ sono gli stati della base di Fourier, $|\alpha_{k|j}|$ è grande $\Leftrightarrow \lambda_j\simeq\frac{2\pi k}{t_0}$. Definend $\overline{\lambda}_k=\frac{2\pi k}{t_0}$ possiamo rinominare il nostro registro $\ket{k}$ ottenendo $$\sum_{j=1}^N\sum{k=0}^{T-1}\alpha_{k|j}\beta_j\ket{\overline{\lambda}_k}\ket{u_j}$$
Aggiungendo un qubit ancillare e ruotando condizionati su $\ket{\overline{\lambda}_k}$ ritorna
$$\sum_{j=1}^N\sum_{k=0}^{T-1}\alpha_{k|j}\beta_j\ket{\overline{\lambda}_k}\ket{u_j}\left(\sqrt{1-\frac{C^2}{\overline{\lambda}_k^2}}\ket{0}+\frac{C}{\overline{\lambda}_k}\ket{1}\right)$$
dove $C=O(\frac{1}{\mathbb{k}})$. Ora annulliamo la stime della fase per de-calcolare $\ket{\overline{\lambda}_k}$. Se la stima di fase è perfetta, avremo $\alpha_{k|j}=1$ se $\overline{\lambda}_k = \lambda_k$ e 0 altrimenti. Assumendo ciò per ora, otteniamo:
$$\sum_{j=1}^N\sum_{k=0}^{T-1}\beta_j\ket{u_j}\left(\sqrt{1-\frac{C^2}{\lambda_k^2}}\ket{0}+\frac{C}{\lambda_k}\ket{1}\right)$$
Per concludere l'inversione misuriamo l'ultimo qubit. Condizionando sull'osservare 1, abbiamo lo stato
$$\sqrt{\frac{1}{\sum_{j=1}^N\frac{C^2|\beta_j|^2}{|\lambda_j|^2}}}\sum_{j=1}^N\beta_j\frac{C}{\lambda_j}\ket{u_j}$$
che corrisponde a $\ket{x}=\sum_{j=1}^n\beta_j\lambda_j^{-1}\ket{u_j}$ a meno di normalizzazione. Possiamo determinare il fattore di normalizzatione dalla probabilità di ottenere 1.

Infine eseguiamo la misurazione $M$ il cui valore atteso $\langle x\:|\:M\:|\:x\rangle$ corrisponde alla feature di $x$ che vogliamo valutare.

### HHL Step by step

L'algoritmo HHL ha 5 componenti principali:
- preparazione dello stato
- QPE
- rotazione del bit ancillare
- QPE inversa (IQPE)

Il registro $b$, dei bit più significativi, è accompagnato da altri due registri.
- Il primo registro è il $c$, perché è relativo al clock nella rotazione controllata del QPE, contiene i valori della fase degli autovalori della matrice $A$ dopo QPE ed è composto da $n$ qubit. Si usa il basis encoding (il numero di fase è codificato nel numero di base), quindi $n$ detta l'accuratezza della memorizzazione. $N = 2^n$
- Il secondo registro è il qubit ancillare $\ket{\:}_a$, che è anche il bit meno significativo. Viene scartato alla fine della computazione.

La matrice $A$, che è Hamiltoniana, può essere scritta come combinazione lineare del prodotto esterno dei suoi autovettori $\ket{u_i}\bra{u_i}$ pesati per il rispettivo autovalore $\lambda_i$ $$A=\sum_{i=0}^{2^{n_b}-1}\lambda_i\ket{u_i}\bra{u_i}$$
Dato che così riscritta è diagonale, la sua inversa è semplicemente $A^{-1}=\sum_{i=0}^{2^{n_b}-1}\lambda_i^{-1}\ket{u_i}\bra{u_i}$

$b$ può essere espresso nella base formata dagli autovettori di $A$ $$\ket{b}=\sum_{j=0}^{2^{n_b}-1}b_j\ket{u_j}$$

Di conseguenza $\langle u_i|u_j\rangle = \delta_{ij}$ e abbiamo $$\ket{x}=A^{-1}\ket{b}=\sum_{i=0}^{2^{n_b}-1}\lambda_i^{-1}b_i\ket{u_i}$$

L'obiettivo di HHL è trovare la soluzione in questa forma e memorizzarla nel registro $b$.

La soluzione non sarà $\lambda_i^{-1}b_i$, che sono le ampiezze nella base degli autovettori, perché noi vogliamo la soluzione nella base computazionale $\ket{0},\ket{1}$. Però misurando nella base computazionale si otterranno le ampiezze corrette, e questo è possibile solo se i qubit non sono in entanglement con altri qubit.

Le condizioni di normalità devono essere rispettate:
$$\sum_{j=0}^{2^{n_b}-1}|b_j|^2=1$$
$$\sum_{i=0}^{2^{n_b}-1}|\lambda_i^{-1}b_i|^2=1$$

#### Preparazione dello stato

Ci sono un totale di $n_b+n+1$ qubit inizializzati come $$\ket{\Psi_0}=\ket{0\ldots0}_b\ket{0\ldots0}_c\ket{0}_a = \ket{0}^{\otimes n_b}\ket{0}^{\otimes n}\ket{0}$$

Il registro $b$ deve essere ruotato in modo da avere le ampiezze corrispondenti ai coefficienti di $b$ $$b=\left(\begin{array}{c}\beta_0\\\beta_1\\\vdots\\\beta_{n_b-1}\end{array}\right)\Leftrightarrow\ket{b}=\beta_0\ket{0}+\beta_1\ket{1}+\ldots+\beta_{n_b-1}\ket{n_b-1}$$

Di conseguenza $$\ket{\Psi_1}=\ket{b}_b\ket{0\ldots0}_c\ket{0}_a$$

#### QPE

QPE è anche un algoritmo di stima degli autovalori. Ha tre componenti:
- superposizione dei qubit di clock con Hadamard gates
- rotazione controllata
- QFT inversa (IQFT)

L'obiettivo è stimare la fase degli autovalori della matrice di rotazione unitaria $U=e^{iAt}$ nel gate controllato $CU$ usato nella QPE. Questo gate codifica $A$ nella sua Hamiltoniana, e ha autovalori unitari (nella forma $e^{i\theta}$) poiché è unitario. Quindi la fase dell'autovalore del gate è proporzionale all'autovalore di $A$. Come risultato, usare QPE in HHL porta ad avere gli autovalori di $A$ codificati nel registro $c$ dopo la QPE (cioè a $\ket{\Psi_4}$), e saranno codificati in basis encoding, non memorizzando gli autovalori esatti.

Nel primo passo della QPE, Hadamard gates vengono applicati ai clock qubits per crearne una superposizione $$\ket{\Psi_2} = (I^{\otimes n_b}\otimes H^{\otimes n}\otimes I)\ket{\Psi_1}= \ket{b}\frac{1}{2^{\frac{n}{2}}}(\ket{0}+\ket{1})^{\otimes n}\ket{0}$$

Nella parte di rotazione controllata, vengono usati i clock qubit nel registro $c$ come controllo e $\ket{b}$ come target. Il numero dei qubit del registro $c$, $n$, determina il numero di gate controllati. I gate sono nella forma $U^r$, con $r$ indice del qubit di clock.
- $\ket{c_{n-1}}$ controllerà il gate $U^{2^{n-1}} = e^{iAt2^{n-1}}$
- $\ket{c_{0}}$ controllerà il gate $U^{2^{0}} = e^{iAt2^{0}} = e^{iAt} = U$

Per capire come funziona, consideriamo $\ket{b}$ come autovettore di $U$ di autovalore $e^{2\pi i\phi}$. L'autovalore così scritto porterà la fase $\phi$ ad essere codificata come basis state nel registro $c$. Di consuegenza $$U\ket{b}=e^{2\pi i\phi}\ket{b}$$
Quando il bit di controllo è $\ket{0}$, $\ket{b}$ non viene modificato. Quando invece è $\ket{1}$, $U$ viene applicato a $\ket{b}$, che è equivalente a moltiplicare per $e^{2\pi i\phi 2^j}$ davanti il qubit $\ket{1}$ del $j$-esimo qubit di clock $\ket{c_j}$.

Di conseguenza dopo i gate $U$ controllati abbiamo 
$$\ket{\Psi_3} = \ket{b}\otimes\left(\frac{1}{2^{\frac{n}{2}}}(\ket{0}+e^{2\pi i\phi 2^{n-1}}\ket{1})\otimes (\ket{0}+e^{2\pi i\phi 2^{n-2}}\ket{1})\otimes \ldots \otimes (\ket{0}+e^{2\pi i\phi 2^{n-n}}\ket{1})\right)\otimes \ket{0}_a =\ket{b}\frac{1}{\sqrt{2}}\sum_{k=0}^{2^n-1}e^{2\pi i\phi k}\ket{k}\ket{0}_a$$

In IQFT abbiamo una matrice di trasformazione $U_I$ contenente le potenze della $N$-esima radice dell'unità
$$\omega_N=e^{\frac{i2\pi}{N}}$$
$$U_I=\frac{1}{\sqrt{N}}\left(\begin{array}{c c c c}\omega_N^{-0\cdot 0}&\omega_N^{-0\cdot 1}&\ldots&\omega_N^{-0\cdot (N-1)}\\\omega_N^{-1\cdot 0}&\omega_N^{-1\cdot 1}&\ldots&\omega_N^{-1\cdot (N-1)}\\\vdots&\vdots&\ddots&\vdots\\\omega_N^{-(N-1)\cdot 0}&\omega_N^{-(N-1)\cdot 1}&\ldots&\omega_N^{-(N-1)\cdot(N-1)} \end{array}\right)$$
$$U_I\ket{k}=\frac{1}{\sqrt{N}}\sum_{j=0}^{N-1}\omega_N^{-jk}\ket{j}$$

Applicando IQFT ad un basis vector, viene ruotato in una superposizione di tutti i basis vector pesati dalle potenze della $N$-esima radice dell'unità.

In HHL, nella parte di QFT, solo i qubit di clock sono modificati.
$$\ket{\Psi_4} = \ket{b}IQFT\left(\frac{1}{\sqrt{2}}\sum_{k=0}^{2^n-1}e^{2\pi i\phi k}\ket{k}\right)\ket{0}_a=$$
$$ = \ket{b}\frac{1}{\sqrt{2}}\sum_{k=0}^{2^n-1}e^{2\pi i\phi k}\:IQFT(\ket{k})\ket{0}_a=$$
$$=\ket{b}\frac{1}{\sqrt{2}}\sum_{k=0}^{2^n-1}e^{2\pi i\phi k}\left(\sum_{y=0}^{2^n-1} e^{-\frac{2\pi iyk}{N}}\ket{y}\right)\ket{0}_a =$$
$$=\ket{b}\frac{1}{\sqrt{2}}\sum_{k=0}^{2^n-1}e^{2\pi ik(\frac{\phi-y}{N})}\ket{y}\ket{0}_a$$

Solo i $\ket{y}$ che soddisfano $\frac{\phi - y}{N} = 0$ avranno un'ampiezza finita pari a $\sum_{k=0}^{2^n-1}e^{0} = 2^n$, altrimenti l'ampiezza è $\sum_{k=0}^{2^n-1}e^{2\pi ik(\frac{\phi-y}{N})}=0$ a causa di interferenza distruttiva. Ignorando gli stati ad ampiezza zero:
$$\ket{\Psi_4}=\frac{1}{2^n}\ket{b}\sum_{k=0}^{2^n-1}e^{2\pi ik\cdot 0}\ket{N\phi}\ket{0}_a = \ket{b}\ket{N\phi}\ket{0}_a$$

Quindi in QPE i qubit di clock sono usati per rappresentare l'informazione sulla fase di $U$, che è $\phi$. L'accuratezza dipende dal numero di qubit $n$. Avendo Hamiltonian encoding, $A$ è legato a $U$ attraverso $U=e^{iAt}$ dove $t$ è il tempo di evoluzione per l'Hamiltoniana. $U$ è diagonale nella base di autovettori $\ket{u_i}$ di $A$. Se $\ket{b} = \ket{u_j}$ allora $U\ket{b}=e^{i\lambda_j t}\ket{u_j}$. Mettendo a equazione $i\lambda_jt = 2\pi i\phi$ otteniamo $\phi=\frac{\lambda_jt}{2\pi}$ e possiamo scrivere
$$\ket{\Psi_4}=\ket{u_j}\ket{N\frac{\lambda_jt}{2\pi}}\ket{0}_a$$
Così facendo gli autovalori di $A$ sono codificati nei qubit di clock (basis encoding). In generale, però, $\ket{b} = \sum_{j=0}^{2^{n_b}-1}b_j\ket{u_j}$ e usando la superposizione
$$\ket{\Psi_4}=\sum_{j=0}^{2^{n_b}-1}b_j\ket{u_j}\ket{N\frac{\lambda_jt}{2\pi}}\ket{0}_a$$
$\lambda_j$ solitamente non sono interi. Scegliamo $t$ così che $\overline{\lambda_j}=N\frac{\lambda_jt}{2\pi}$ siano interi. Di conseguenza, i valori codificati di $\overline{\lambda_j}$ sono diversi da quelli di $\lambda_j$. Possiamo quindi riscrivere come
$$\ket{\Psi_4}=\sum_{j=0}^{2^{n_b}-1}b_j\ket{u_j}\ket{\overline{\lambda_j}}\ket{0}_a$$

#### Rotazione controllata e misurazione del qubit ancillare

Il prossimo passo è ruotare il qubit ancillare $\ket{0}_a$ sulla base degli autovalori codificati nel registro $c$, in modo che
$$\ket{\Psi_5}=\sum_{j=0}^{2^{n_b}-1}b_j\ket{u_j}\ket{\overline{\lambda_j}}\left(\sqrt{1-\frac{C^2}{\overline{\lambda_j}^2}}\ket{0}_a+\frac{C}{\overline{\lambda_j}}\ket{1}_a\right)$$
con $C$ costante. Questo è l'obiettivo, vedremo perché è utile.

Quanto il qubit ancillare è misurato, la funzione d'onda (wavefunction) del qubit ancillare collassa a $\ket{0}$ oppure a $\ket{1}$. Se viene misurato $\ket{0}$, il risultato è scartato e la computazione viene ripetuta, questo finché non si misura $\ket{1}$. Di conseguenza la funzione d'onda finale d'interesse è
$$\ket{\Psi_6}=\frac{\displaystyle 1}{\displaystyle \sqrt{\sum_{j=0}^{2^{n_b}-1}\left|\frac{b_jC}{\overline{\lambda_j}}\right|^2}}\sum_{j=0}^{2^{n_b}-1}b_j\ket{u_j}\ket{\overline{\lambda_j}}\frac{C}{\overline{\lambda_j}}\ket{1}_a$$
con il fattore iniziale dovuto alla normalizzazione dopo la misurazione. Dato che $|\frac{b_jC}{\overline{\lambda_j}}|^2$ è la probabilità di misurare $\ket{1}$ sul bit ancillare, $C$ dev'essere scelto in modo da essere il più grande possibile. Il risultato somiglia a $\ket{x}$ che stiamo cercando, ma possiamo ottenere il risultato corretto solo misurando nella base di autovettori $\ket{u_j}$ e non nella base computazionale $\ket{0}, \ket{1}$. Però il registro $b$ è in entanglement con i qubit di clock $\ket{\overline{\lambda_j}}$, quindi non possiamo fattorizzare il risultato in un prodotto tensoriale tra i registri $c$ e $b$. Di conseguenza non possiamo convertire il registro $b$ nella base computazionale, dobbiamo de-computare lo stato così che dia il giusto risultato, con i registri $b$ e $c$ rimossi dall'entanglement.

Questa misurazione del qubit ancillare può e solitamente è fatta dopo la de-computazione. Però, dato che questo qubit non viene ulteriormente toccato, misurarlo prima fornisce lo stesso risultato.

#### De-computazione: IQPE

Per prima cosa si applica QFT sui qubit di clock

$$\ket{\Psi_7}=\frac{1}{ \sqrt{\sum_{j=0}^{2^{n_b}-1}|\frac{b_jC}{\overline{\lambda_j}}|^2}}\sum_{j=0}^{2^{n_b}-1}b_j\ket{u_j}\:QFT(\ket{\overline{\lambda_j}})\frac{C}{\overline{\lambda_j}}\ket{1}_a=$$
$$=\frac{1}{ \sqrt{\sum_{j=0}^{2^{n_b}-1}|\frac{b_jC}{\overline{\lambda_j}}|^2}}\sum_{j=0}^{2^{n_b}-1}b_j\ket{u_j}\left(\frac{1}{\sqrt{2^n}}\sum_{y=0}^{2^n-1}e^{\frac{2\pi iy\overline{\lambda_j}}{N}}\ket{y}\right)\frac{C}{\overline{\lambda_j}}\ket{1}_a$$

Dopo di ciò, rotazione controllate inverse del registro $b$ controllate dai qubit di clock sono applicate, con $U^{-1}=e^{-iAt}$. Come per il processo controllato precedentemente, se l'$r$-esimo qubit di clock è $\ket{0}$ allora $\ket{u_j}$ non viene toccato, altrimenti $(U^{-1})^{2^r}$ è applicato a $\ket{u_j}$. Questo è equivalente a moltiplicare per $e^{-i\lambda_jty}$ se il registro $c$ è $\ket{y}$, poiché $2\pi i\phi=i\lambda_j t$. Quindi
$$\ket{\Psi_8}=\frac{1}{\sqrt{2^n}\sqrt{\sum_{j=0}^{2^{n_b}-1}|\frac{b_jC}{\overline{\lambda_j}}|^2}}\sum_{j=0}^{2^{n_b}-1}\frac{b_jC}{\overline{\lambda_j}}\ket{u_j}\left(\sum_{y=0}^{2^n-1}e^{-i\lambda_jty}e^{\frac{2\pi iy\overline{\lambda_j}}{N}}\ket{y}\right)\ket{1}_a=$$
Dato che $\overline{\lambda_k}=\frac{N\lambda_j t}{2\pi}$, i due termini esponenziali si annullano a vicenda
$$=\frac{1}{\sqrt{2^n} \sqrt{\sum_{j=0}^{2^{n_b}-1}|\frac{b_jC}{\overline{\lambda_j}}|^2}}\sum_{j=0}^{2^{n_b}-1}\frac{b_jC}{\overline{\lambda_j}}\ket{u_j}\sum_{y=0}^{2^n-1}\ket{y}\ket{1}_a=$$
$$=\frac{1}{\left(\sqrt{2^n}\right)^{\frac{Nt}{2\pi}} \sqrt{\sum_{j=0}^{2^{n_b}-1}|\frac{b_jC}{\overline{\lambda_j}}|^2}}\sum_{j=0}^{2^{n_b}-1}\frac{b_jC}{\lambda_j}\ket{u_j}\sum_{y=0}^{2^n-1}\ket{y}\ket{1}_a=$$
$$=\frac{1}{\sqrt{2^n} \sqrt{\sum_{j=0}^{2^{n_b}-1}|\frac{b_jC}{\lambda_j}|^2}}\ket{x}\sum_{y=0}^{2^n-1}\ket{y}\ket{1}_a$$

I qubit di clock e il registro $b$ sono ora fuori dall'entanglement e il registro $b$ contiene $\ket{x}$. Applicando Hadamard ai qubit di clock abbiamo finalmente
$$\ket{\Psi_9} = \frac{1}{\sqrt{\sum_{j=0}^{2^{n_b}-1}|\frac{b_jC}{\lambda_j}|^2}}\sum_{j=0}^{2^{n_b}-1}\frac{b_jC}{\lambda_j}\ket{u_j}\ket{0}^{\otimes n}\ket{1}_a =$$
$$=\frac{1}{\sqrt{\sum_{j=0}^{2^{n_b}-1}|\frac{b_jC}{\lambda_j}|^2}}\ket{x}_b\ket{0}^{\otimes n}\ket{1}_a$$

Se $C\in \mathbb{R}$, usando il fatto che $\sum_{i=0}^{2^{n_b}-1}|\lambda_i^{-1}b_i|^2=1$ abbiamo che
$$\ket{\Psi_9}=\frac{1}{\sqrt{\sum_{j=0}^{2^{n_b}-1}|\frac{b_j}{\lambda_j}|^2}}\ket{x}_b\ket{0}^{\otimes n}\ket{1}_a=$$
$$=\ket{x}_b\ket{0}_c^{\otimes n}\ket{1}_a$$

Con la soluzione $\ket{x}$ correttamente memorizzata nel registro $b$.

#### Esempio Numerico
Gli elementi di questo esempio sono
$$A=\left(\begin{array}{cc}1&-\frac{1}{3}\\-\frac{1}{3}&1\end{array}\right)$$
$$b=\left(\begin{array}{c}0\\1\end{array}\right)$$
Gli autovettori di $A$ sono
$$u_0=\left(\begin{array}{c}\frac{-1}{\sqrt{2}}\\\frac{-1}{\sqrt{2}}\end{array}\right)\:\:\:u_1=\left(\begin{array}{c}\frac{-1}{\sqrt{2}}\\\frac{1}{\sqrt{2}}\end{array}\right)$$
di autovalori $$e_0=\frac{2}{3}\:\:\:e_1=\frac{4}{3}$$
Usiamo la basis encoding per gli autovalori, nella base formata dai qubit di clock: 2 qubit usati per codificare $e_0\mapsto\ket{01}$ e altri 2 per $e_1\mapsto\ket{10}$, per mantenere $\frac{e_0}{e_1}=2$. Questo significa che $\overline{\lambda_0}=1$ e $\overline{\lambda_1}=2$ e cioè che $\ket{\overline{\lambda_0}}=\ket{01}$ e $\ket{\overline{\lambda_1}}=\ket{10}$ dando codifica perfetta con $n=2\Leftrightarrow N=4$. Quindi viene scelto $t=\frac{3\pi}{4}$ dato che $\overline{\lambda_0}=\frac{N\lambda_jt}{2\pi}$

$b$ è un vettore complesso di 2 dimensioni, per cui è codificabile con 1 qubit $\Rightarrow n_b=1$

La soluzione che verrà trovata è $$x=\left(\begin{array}{c}\frac38\\\frac98\end{array}\right)$$